In [1]:
import torch
from torch_geometric.loader import DataLoader
from data.graph_dataset import OneDDatasetBuilder, OneDDatasetLoader, normalize, dataset_to_loader, batchgraph_generation_wise, batchgraph
import matplotlib.pyplot as plt

In [2]:
class objectview(object):
    def __init__(self, d) -> None:
        self.__dict__ = d
    def setattr(self, attr_name, attr_value):
        self.__dict__[attr_name] = attr_value

args = objectview({
    'n_fields': 1,
    'n_meshfields': (13, 0),
    'n_timesteps': 201,
    'n_previous_timesteps': 1,
    'aggr': 'sum',
    'alpha': 1.0,
    'batch_size': 100,
    'timestep': 201,
    'timeslice_hops': 0,
    'timeslice_steps': 1,
    'n_data_per_batch': 50,
    'forward_sequence': False,
    'plot': False
})

In [3]:
# dataset = OneDDatasetBuilder(
#     raw_dir='./datasets',
#     root_dir='./pre_data',
#     sub_dir='processed',
#     subjects='all',
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type = torch.float32,
#     readme='edge_index(2xn_edge), node_attr(n_nodex10), pressure+flowrate(n_nodex201)'
# )

In [4]:
# datasets = OneDDatasetLoader(
#     root_dir='./pre_data',
#     sub_dir='processed',
#     subjects='all',
#     time_names=[str(i).zfill(3) for i in range(201)],
#     data_type = torch.float32
# )

# dataset = normalize(
#     dataset=datasets,
#     sub_dir='normalized',
#     scaler_dict={
#         'node_attr': ('minmax_scaler', 0, None),
#         'pressure': ('minmax_scaler', None, None),
#         'flowrate': ('minmax_scaler', None, None)
#     }
# )

In [5]:
datasets = OneDDatasetLoader(
    root_dir='./pre_data',
    sub_dir='normalized',
    subjects='all',
    time_names=[str(i).zfill(3) for i in range(201)],
    data_type = torch.float32
)

dataset = batchgraph_generation_wise(
    datasets,
    sub_dir='batched',
    batch_gens=[[0,7], [8, 50]],
    timestep=None,
    timeslice_hops=args.timeslice_hops,
    timeslice_steps=args.timeslice_steps
)

Processing...
Done!


TorchGraphData(node_attr=[150, 13], edge_index=[2, 149], pressure=[150, 201], flowrate=[150, 201])
TorchGraphData(node_attr=[59437, 13], edge_index=[2, 59283], pressure=[59437, 201], flowrate=[59437, 201])
TorchGraphData(node_attr=[128, 13], edge_index=[2, 127], pressure=[128, 201], flowrate=[128, 201])
TorchGraphData(node_attr=[59617, 13], edge_index=[2, 59488], pressure=[59617, 201], flowrate=[59617, 201])
TorchGraphData(node_attr=[123, 13], edge_index=[2, 122], pressure=[123, 201], flowrate=[123, 201])
TorchGraphData(node_attr=[59437, 13], edge_index=[2, 59316], pressure=[59437, 201], flowrate=[59437, 201])
TorchGraphData(node_attr=[148, 13], edge_index=[2, 147], pressure=[148, 201], flowrate=[148, 201])
TorchGraphData(node_attr=[59488, 13], edge_index=[2, 59336], pressure=[59488, 201], flowrate=[59488, 201])
TorchGraphData(node_attr=[140, 13], edge_index=[2, 139], pressure=[140, 201], flowrate=[140, 201])
TorchGraphData(node_attr=[59751, 13], edge_index=[2, 59613], pressure=[59751,

Processing...
Done!


In [6]:
data = torch.load('./pre_data/batched/batched_id.pt')

In [7]:
unique_values, counts = torch.unique(data, return_counts=True)

print("Values:", unique_values)
print("Counts:", counts)

Values: tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27.,
        28., 29., 30., 31., 32., 33., 34.], dtype=torch.float64)
Counts: tensor([257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257,
        257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257, 257,
        257, 257, 257, 257, 257, 257, 257])


In [ ]:
x = next(iter(train_set)).cuda()
x_trunk = torch.linspace(-1, 1, 201).cuda()
x_trunk = x_trunk[:, None]
model = Net1D(n_branch=13, width=1000, depth=3, p=512, act=torch.relu).cuda()
output = model(x.node_attr, x.edge_index, x_trunk)

myloss = LpLoss(size_average=True)
loss = myloss(output.to(torch.float32), x.pressure.to(torch.float32))
print(loss.item())